In [1]:
import numpy as np
import plotly.express as ex
import plotly.graph_objects as go
from plotly.subplots import make_subplots


SEED = 2020

In [8]:
class RBF_Network:
    def __init__(self, n, inputs, variance=1, add_noise=False, useDelta=False, eta=0.1, useSqr=False):
        """
        weights : (n, 1)
        mean : (n, 1)
        variance : (n, 1)
        """
        
        np.random.seed(SEED)

        self.n = n
        self.add_noise = add_noise
        self.useDelta = useDelta
        self.eta = eta
        self.useSqr=useSqr

        # set position and width for each node
        self.mean = self.mu_random_choice(inputs)
        self.std = np.ones((n,variance))

        self.W = np.random.randn(self.n, 1)
        
        self.phi_of_x = np.reshape(np.array(self.mean), (1,-1))
        self.total_error = []
        self.residual_error = []
        self.f_of_x = []

    def mu_random_choice(self, inputs):
        """
        x = Input patterns
        n = number of nodes
        """
        return np.random.choice(x.ravel(), (self.n,1))

    def train_network(self, epochs, train_data, train_label, test_data=[], test_label=[]):
        """
        train_data, test_data : (N,1)
        train_label test_label : (N,1)
        """
        
        for e in range(0, epochs):
            if (self.add_noise):
                train_label = self.add_noise(train_label)
                train_label = self.add_noise(test_label)
            if (self.useDelta):
                self.delta_learning(train_data, train_label)
            else:
                self.lsqr(train_data, train_label)
            
            if (len(test_data) > 0):
                self.residual_error.append(self.validate(test_data, test_label))


    def add_noise(self, x, std=0.1):
        noise = np.random.normal(0,std,(x.shape[0],1))
        x += noise
        return x
    
    def forward(self, x):
        self.phi_of_x = np.array([self.activation_fxn(x, i) for i in range(self.n)]).T
        self.f_of_x= self.phi_of_x @ self.W

    def activation_fxn(self, x, i):
        _x = x.ravel()
        _mu = self.mean[i].ravel()
        _std = self.std[i].ravel()
        return np.exp((-(_x-_mu)**2) / 2*(_std**2))

    def calc_total_error(self, label):
        """
        MSE
        """
        self.total_error.append(np.mean((self.f_of_x-label)**2))

    def update_weights(self, label):
        self.W, _,_,_ = np.linalg.lstsq(self.phi_of_x, label.ravel())
        self.W.reshape((-1,1))

    def lsqr(self, x, y):
        self.forward(x)
        self.update_weights(y)

    def delta_learning(self, x, y):
        for e in range(epochs):
            for i in range(len(x)):
                self.forward(x[i])
                error = y[i] - self.f_of_x
                self.W += (self.eta * error)[0] * self.phi_of_x.T
         
    def validate(self, test_data, test_label, _plot=False):
        self.forward(test_data)
        if (self.useSqr):
            output = np.sign(self.f_of_x)
        else:
            output = self.f_of_x

        if _plot == True:
            fig = make_subplots(rows=1, cols=2, subplot_titles=("Validation with n={}".format(self.n), "Residual error"))
            fig.add_trace(go.Scatter(x=np.arange(len(self.residual_error)), y=self.residual_error, mode="lines", name="Residual error"), row=1, col=2)
            fig.add_trace(go.Scatter(x=test_data.ravel(), y=test_label.ravel(), mode="lines", name="Original"), row=1, col=1)
            fig.add_trace(go.Scatter(x=test_data.ravel(), y=output.ravel(), mode="lines", name="Approximated"), row=1, col=1)

            fig.show()

        return np.mean(np.abs(output-test_label.ravel())) 
        
    def init_cl(self, x, epochs, eta, useBias=False, bias_w=1e-2):
        """
        Run CL-algorithm for positioning the nodes
        Use bias to eliminate dead nodes
        """

        b = np.ones((self.n,1)) * 1/self.n # Bias for each node
        wins = np.zeros((self.n,1)) # Counting the number of wins for each ndoe

        for i in range(epochs):
            np.random.shuffle(x)
            for x_i in x:
                error = x_i - self.mean
                dist = error**2
                
                if (useBias):
                    dist -= bias_w * b

                winner = np.argmin(dist)
                self.mean[winner,:] += eta*error[winner]
                wins[winner]+=1       

                if (useBias):
                    b = 1/self.n - wins/(winner+1)         

                







In [14]:
def data_gen(start, end, step=0.1):
    x = np.arange(start, end, step).reshape((-1,1))
    sin_data = np.sin(2*x).reshape((-1,1))
    sqr_data = np.sign(sin_data)
    return x, sin_data, sqr_data

def plot_fx_n_errror(patterns, targets, approximation, res_error, titles):
    fig = make_subplots(rows=1, cols=2, subplot_titles=(titles[0], titles[1]))
    fig.add_trace(go.Scatter(x=np.arange(len(res_error)), y=res_error, mode="lines", name="Residual error"), row=1, col=2)
    fig.add_trace(go.Scatter(x=x.ravel(), y=y.ravel(), mode="lines", name="Original"), row=1, col=1)
    fig.add_trace(go.Scatter(x=x.ravel(), y=approximation.ravel(), mode="lines", name="Approximated"), row=1, col=1)

    fig.show()


In [4]:
x, sin_data, sqr_data = data_gen(0, 2*np.pi)

fig = go.Figure().add_trace(go.Scatter(x=x.ravel(), y=sin_data.ravel(), mode="lines", name="sin(x)"))
fig.add_trace(go.Scatter(x=x.ravel(), y=sqr_data.ravel(), mode="lines", name="sqr(x)"))
fig.update_layout(title="Functions to approximate")
fig.show()

## Quick test of the model

Sin(x)

In [11]:
x, sin_data, sqr_data = data_gen(0, 2*np.pi)
t_x, t_sin_data, t_sqr_data = data_gen(0.05, 2*np.pi)

n = 10
epochs = 2 # Actually only one iteration is enough

RBFNet = RBF_Network(n, x)
# RBFNet.init_cl(x, 50, 1e-2, useBias=True) # Uncomment this to use CL initialization
RBFNet.train_network(epochs, x, sin_data, t_x, t_sin_data)
res_error = RBFNet.validate(t_x, t_sin_data, _plot=True)

print(res_error)

0.010380313768479206


Square(2x)

In [12]:
x, sin_data, sqr_data = data_gen(0, 2*np.pi)
t_x, t_sin_data, t_sqr_data = data_gen(0.05, 2*np.pi, step=0.05)

n = 10
epochs = 2

RBFNet = RBF_Network(n, x, useSqr=True)

# RBFNet.init_cl(x, 50, 1e-2) # Uncomment this to use CL initialization
RBFNet.train_network(epochs, x, sqr_data, t_x, t_sqr_data)
res_error = RBFNet.validate(t_x, t_sqr_data, _plot=True)

print(res_error)

0.0


Delta learining

In [17]:
x, sin_data, sqr_data = data_gen(0, 2*np.pi)
t_x, t_sin_data, t_sqr_data = data_gen(0.05, 2*np.pi)

n = 10
epochs = 50 # Actually only one iteration is enough

RBFNet = RBF_Network(n, x, eta=1e-1, useDelta=True)
RBFNet.init_cl(x, 100, 1e-2) # Uncomment this to use CL initialization
RBFNet.train_network(epochs, x, sin_data, t_x, t_sin_data)
res_error = RBFNet.validate(t_x, t_sin_data, _plot=True)

print(res_error)

0.7165005017109901
